In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from scipy import misc
import sys
import os
import tensorflow as tf
import numpy as np
import facenet
import align.detect_face
import random
from time import sleep

import math
import pickle
from sklearn.svm import SVC

In [7]:
margin = 44
gpu_memory_fraction = 1.0
detect_multiple_faces = False

In [8]:
seed = 666
image_size = 160
do_prewhiten = True
model = '/home/topica/workspace/facenet/src/ckpt/20180402-114759/'
classifier_filename = '/home/topica/workspace/facenet/src/model_topica_weight.pkl'
class_name = ['0','1','2','3','4','5','6','7','8','9','10']
image_path = '/home/topica/workspace/facenet/dataset_topica/0/face_2_334.png'

In [9]:
sleep(random.random())

print('Creating networks and loading parameters')

with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)

minsize = 20 # minimum size of face
threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
factor = 0.709 # scale factor

try:
    img = misc.imread(image_path)
except (IOError, ValueError, IndexError) as e:
    errorMessage = '{}: {}'.format(image_path, e)
    print(errorMessage)
else:
    if img.ndim<2:
        print('Unable to align "%s"' % image_path)
    if img.ndim == 2:
        img = facenet.to_rgb(img)
    img = img[:,:,0:3]

    bounding_boxes, _ = align.detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
    nrof_faces = bounding_boxes.shape[0]
    print(img)
    if nrof_faces>0:
        det = bounding_boxes[:,0:4]
        det_arr = []
        result_face = []
        img_size = np.asarray(img.shape)[0:2]
        if nrof_faces>1:
            if detect_multiple_faces:
                for i in range(nrof_faces):
                    det_arr.append(np.squeeze(det[i]))
            else:
                bounding_box_size = (det[:,2]-det[:,0])*(det[:,3]-det[:,1])
                img_center = img_size / 2
                offsets = np.vstack([ (det[:,0]+det[:,2])/2-img_center[1], (det[:,1]+det[:,3])/2-img_center[0] ])
                offset_dist_squared = np.sum(np.power(offsets,2.0),0)
                index = np.argmax(bounding_box_size-offset_dist_squared*2.0) # some extra weight on the centering
                det_arr.append(det[index,:])
        else:
            det_arr.append(np.squeeze(det))

        for i, det in enumerate(det_arr):
            det = np.squeeze(det)
            bb = np.zeros(4, dtype=np.int32)
            bb[0] = np.maximum(det[0]-margin/2, 0)
            bb[1] = np.maximum(det[1]-margin/2, 0)
            bb[2] = np.minimum(det[2]+margin/2, img_size[1])
            bb[3] = np.minimum(det[3]+margin/2, img_size[0])
            cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
            scaled = misc.imresize(cropped, (image_size, image_size), interp='bilinear')
#             result_face.append(scaled)
            print(type(scaled),scaled.shape)
            print(type(bb), bb.shape)
    else:
        print('Unable to align "%s"' % image_path)

Creating networks and loading parameters
[[[130 138 128]
  [127 135 125]
  [120 128 118]
  ...
  [129 135 124]
  [129 135 124]
  [128 134 125]]

 [[131 139 129]
  [127 135 125]
  [119 127 117]
  ...
  [128 134 123]
  [127 133 122]
  [126 132 123]]

 [[131 139 129]
  [127 135 125]
  [118 126 116]
  ...
  [126 132 121]
  [125 131 120]
  [124 130 121]]

 ...

 [[203 199 191]
  [204 198 191]
  [201 195 188]
  ...
  [255 255 255]
  [238 253 255]
  [104 120 131]]

 [[204 200 192]
  [204 198 191]
  [200 194 187]
  ...
  [255 255 255]
  [244 255 255]
  [193 208 222]]

 [[203 199 191]
  [204 198 191]
  [200 194 187]
  ...
  [254 255 255]
  [232 245 255]
  [148 163 177]]]
<class 'numpy.ndarray'> (160, 160, 3)
<class 'numpy.ndarray'> (4,)


/home/topica/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/home/topica/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:57: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [10]:
 with tf.Graph().as_default():
        with tf.Session() as sess:   
            np.random.seed(seed=seed)
            
            # Load the model
            print('Loading feature extraction model')
            facenet.load_model(model)
            
            # Get input and output tensors
            images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
            embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
            embedding_size = embeddings.get_shape()[1]
            
            # Run forward pass to calculate embeddings
            print('Calculating features for images')
            emb_array = np.zeros((1, embedding_size))
            if do_prewhiten:
                img = facenet.prewhiten(scaled)
            img = np.reshape(img,(1, image_size, image_size,3))
            feed_dict = { images_placeholder:img, phase_train_placeholder:False }
            emb_array[0:1,:] = sess.run(embeddings, feed_dict=feed_dict)
            
            classifier_filename_exp = os.path.expanduser(classifier_filename)
            print(classifier_filename_exp)


         
            # Classify images
            print('Testing classifier')
            with open(classifier_filename_exp, 'rb') as infile:
                (model, class_names) = pickle.load(infile)

            print('Loaded classifier model from file "%s"' % classifier_filename_exp)

            predictions = model.predict_proba(emb_array)
            print(predictions)
            best_class_indices = np.argmax(predictions, axis=1)
            best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices]
            print(' %s: %.3f' % (class_names[best_class_indices[0]], best_class_probabilities[0]))

#             accuracy = np.mean(np.equal(best_class_indices, labels))
#             print('Accuracy: %.3f' % accuracy)


Loading feature extraction model
Model directory: /home/topica/workspace/facenet/src/ckpt/20180402-114759/
Metagraph file: model-20180402-114759.meta
Checkpoint file: model-20180402-114759.ckpt-275
INFO:tensorflow:Restoring parameters from /home/topica/workspace/facenet/src/ckpt/20180402-114759/model-20180402-114759.ckpt-275
Calculating features for images
/home/topica/workspace/facenet/src/model_topica_weight.pkl
Testing classifier
Loaded classifier model from file "/home/topica/workspace/facenet/src/model_topica_weight.pkl"
[[0.83377208 0.01617437 0.00856499 0.0087578  0.01660836 0.03411879
  0.01905403 0.03210592 0.01309386 0.0049651  0.01278471]]
 0: 0.834


In [12]:
scaled = np.empty((160, 160, 3))
bb = np.empty((4,))

In [16]:
scaled.shape

(160, 160, 3)